In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv', index_col='id')

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
import seaborn as sns

In [ ]:
for i in range(1,15):
    sns.lmplot('cont'+str(i), 'target', train, fit_reg=False)

In [ ]:
train[train['target'] <= 4.5]

In [ ]:
train.drop([241352, 284103, 300936, 307139, 355831], axis = 0, inplace=True)
train[train['target'] <= 4.5]

In [ ]:
for i in range(1, 15):
    sns.lmplot('cont'+str(i), 'target', train, fit_reg=False)

In [ ]:
from sklearn.model_selection import train_test_split
target = train['target']
train.drop('target', axis=1, inplace=True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.15, random_state=0)

In [ ]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(tree_method='gpu_hist',learning_rate=0.07, max_depth=15, random_state=0, n_estimators=2000, n_jobs=-1)
lgbm.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
lgbm.score(X_valid, y_valid)

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(learning_rate=0.005, max_depth=8, n_estimators=6000, n_jobs=-1, tree_method='gpu_hist', random_state=3)
xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
xgb.score(X_valid, y_valid)

In [ ]:
from sklearn.ensemble import VotingRegressor
ensemble = VotingRegressor(estimators=[("xgb", xgb), ("lgbm", lgbm)], weights=[1.1,1])
ensemble.fit(X_train, y_train)
ensemble.score(X_valid, y_valid)

In [ ]:
model = ensemble.fit(train, target)
model.score(train, target)

In [ ]:
pred = model.predict(test)
pred

In [ ]:
id = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')['id']

In [ ]:
output = pd.DataFrame({'id': id, 'target': pred})
output.to_csv("submission.csv", index=False)
print("saved")